# Demonstration of hypothesis testing

This notebook demonstrates Frequentist and Bayesian approaches to hypothesis testing.

In [ ]:
import numpy as np
from scipy import stats
from scipy import special
from matplotlib import pyplot as plt

## Problem

Sun-like stars have a binary fraction of around 50% ($f_\odot=0.5$). We observe a sample of $N$ stars and wish to know whether the binary fraction is consistent with $f=f_\odot=0.5$ or whether it differs. In particular, we have reason to suspect that $f>0.5$ for the kind of stars we are studying.

Assume we can always detect a binary companion if present.

We observe $N=30$ stars, and find that $k=20$ have binary companions. Is this consistent with $f_\odot=0.5$?

## Frequentist approach 1: the binomial test

Our survey is an example of a *binomial* experiment: a set of $N$ independent trials each of which has a fixed probability of success $f$. The number of binaries detected, $K$, then follows a binomial distribution with parameters $N$ and $f$. This has a probability mass function

$P(K=k) = \frac{N!}{k!(N-k)!}f^{k}(1-f)^{N-k}$

We can now state our null and alternative hypotheses mathematically: 
 - $H_0$: $K\sim\mathrm{Binomial}(N,f)$ where $f=0.5$
 - $H_1$: $f>0.5$

We choose $\alpha=0.05$ as our significance level.

Can we reject $H_0$?

As we suspect $f>0.5$, we conduct a one-tailed test. Let's calculate the PMF for our binomial distribution

In [ ]:
k = np.arange(0,31)
N = 30
f = 0.5
alpha=0.05
k_obs = 20
# brute-force this. Note that factorials can get BIG and you'll want to be more clever if N gets too large

pmf = special.factorial(N)/(special.factorial(k)*special.factorial(N-k)) * f**k * (1-f)**(N-k)

check our numerical accuracy

In [ ]:
print(np.sum(pmf))
pmf = pmf/np.sum(pmf)
cdf = np.cumsum(pmf)
cdf = np.hstack([[0],cdf]) # Bet you didn't see this coming!!

In [ ]:
fig,ax = plt.subplots()
ax.bar(k,pmf,color='pink',label='pmf')
ax.set_ylim(0,.15)
ax.set_xlabel('k')
ax.set_ylabel('pmf')
ax2 = ax.twinx()
plt.plot(k,cdf[:-1],c='k',label='cdf')
ax2.set_ylim(0,1)
ax2.set_ylabel('cdf')
ax2.plot([0,N],[1-alpha,1-alpha],'-.',label='$alpha=0.05$',color='blue')
ax2.plot([k_obs,k_obs],[0,1],c='gray',label='$k=20$')
plt.legend()
plt.show()

We see that $k=20$ lies out in the tail of the distribution with the cdf $>1-\alpha=0.95$. We can quantify this a little more by tabulating the cumulative distribution function around that region:

In [ ]:
print('k      cdf')
for i in range(10):
    print(k[i + 15],cdf[i + 15])

print(f'The p-value is {1-cdf[20]}')

Any $k\ge20$ would be sufficient to reject the null hypothesis

What about if we wanted to look for any deviation from $f=0.5$, not just $f>.0.5$? Then we would use a two-tailed test, and the 5% of the area of the pmf gets split between the two tails

In [ ]:
fig,ax = plt.subplots()
ax.bar(k,pmf,color='pink',label='pmf')
ax.set_ylim(0,.15)
ax.set_xlabel('k')
ax.set_ylabel('pmf')
ax2 = ax.twinx()
plt.plot(k,cdf[:-1],c='k',label='cdf')
ax2.set_ylim(0,1)
ax2.set_ylabel('cdf')
ax2.plot([0,N],[1-alpha/2,1-alpha/2],'-.',label='$alpha=0.05$',color='blue')
ax2.plot([k_obs,k_obs],[0,1],c='gray',label='$k=20$')
ax2.plot([0,N],[alpha/2,alpha/2],'-.',color='blue')
plt.legend()
plt.show()

Now we no longer have a significant result!

## Frequentist approach 2: the $\chi^2$ test

We can also treat this with the $\chi^2$ test. We have two 'bins' of data --- binary and not binary --- and under our null hypothesis we would expect $E_1=N\times f=15$ counts in the binary bin and $E_2=N\times(1-f)=15$ in the not binary bin.

Observed counts are $O_1=20$ and $O_2=10$.

We have a test statistic $\chi^2 = \sum_{i=1}^2 \frac{(O_i-E_i)^2}{E_i}$

In [ ]:
chi2 = (k_obs-(N*f))**2 / (N*f) + ((N-k_obs)-(N*(1-f)))**2 / (N*(1-f))

This should follow a $\chi^2$ distribution with 1 degree of freedom (2 bins - 1), which you can access via SciPy

In [ ]:
chi2dist = stats.chi2(1) # creates a chi2 object with 1 dof

x = np.linspace(0,10,1001)
plt.plot(x,chi2dist.cdf(x),label='cdf') # this accesses the cumulative distribution function
c95 = chi2dist.ppf(.95) # the "percent point function": use this to get the critical values
plt.plot([x[0],x[-1]],[0.95,0.95],label='alpha=0.05')
plt.plot([c95,c95],[0,1],label='critical value')
plt.plot([chi2,chi2],[0,1],label='observed value')
plt.xlabel('chi2')
plt.ylabel('cdf')
plt.legend()
plt.show()

On this test, we again don't get a significnat result

## Digression on $\chi^2$ degrees of freedom

The choice of $\nu=n-1$ for the degrees of freedom may seem curious. Let's test it (demonstrating the concept of a sampling distribution in the process)

In [ ]:
# set up a pseudo-random number generator
rng = np.random.default_rng(376452)

# a large sample to avoid small number stats and discretisation effects. We'll set up 1e5 samples each of size 1e4
size = 10000
n_samples = 100000
O1 = rng.binomial(size,f,n_samples) #I could have done this earlier, but wanted to show an explicit calculation
O1 = np.vstack([O1,size-O1]) # simulated bins of binaries and non-binaries
E1 = np.outer(np.array([size*f,size*(1-f)]),np.ones(n_samples))
chisq1 = np.sum((O1-E1)**2/E1,0) # get the chi2 for each 1e5 samples
khisq1 = stats.chi2(1) # the theoretical distribution

In [ ]:
x=np.linspace(0,10)
plt.hist(chisq1,bins=x,density=True,label='Monte-Carlo samples')
plt.plot(x,khisq1.pdf(x),label='theoretical, 1dof') # scipy can give us the pdf too
plt.legend()
plt.show()

A pretty good match! Here we fixed the number of stars per sample at 1e4. This seems to validate the intuition that we fix the total number, so when we specify the count in one bin we also fix that in the other. Hence we have 1 dof, not 2

But what happens if we allow each bin to be populated independently? Let's make a set of Poisson samples where binary plus non-binary does not necessarily add to 1e4...

In [ ]:
O2 = np.vstack([rng.poisson(size*f,n_samples),rng.poisson(size*(1-f),n_samples)])
n2 = np.sum(O2,0)
plt.hist(n2)
plt.title('Different number in each sample')
plt.show()

In [ ]:
E2 = np.vstack([n2*f,n2*(1-f)])
chisq2 = np.sum((O2-E2)**2/E2,0)
khisq2 = stats.chi2(2)

In [ ]:
x=np.linspace(0,10)
plt.hist(chisq2,bins=x,density=True,label='Monte-Carlo samples')
plt.plot(x,khisq1.pdf(x),label='theoretical, 1dof')
plt.plot(x,khisq2.pdf(x),label='theoretical, 2dof')
plt.legend()
plt.show()

Curiously, this still follows the $\chi^2$ distribution with one degree of freedom

## Bayesian approach

Our background information $I$ specifies two models:
 - $M_1$: $K\sim\mathrm{Binomial}(N,f)$ with $f=0.5$
 - $M_2$: $K\sim\mathrm{Binomial}(N,f)$ with a uniform prior on $f$ from 0 to 1

We can easily compute the evidence/marginal likelihood for $M_1$, $P(D|M_1,I)=\frac{N!}{k!(N-k)!}f^{k}(1-f)^{N-k}$

In [ ]:
Evidence1 = pmf[k_obs]
print(Evidence1)

For $M_2$ we have to work harder and marginalise (integrate) over the free parameter:

$P(D|M_2,I) = \int_0^1P(D|f,M_2,I)P(f|M_2,I)df$ = 1/(N+1) after some algebra

In [ ]:
Evidence2 = 1/(N+1)
print(Evidence2)

Very similar! Now we have our Bayes Factor

In [ ]:
Bayes = Evidence2/Evidence1
logBayes = np.log10(Bayes)
print(f'Bayes Factor: {Bayes}')
print(f'log scale:    {logBayes*10} db')

*I.e.,* no strong evidence